In [1]:
%load_ext autoreload
%autoreload 2
from glob import glob
import os
from pathlib import Path
import re
import sys
import yaml
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename, logger_func
from func.ml_utils import get_factorize_feature
from ieee_train import eval_train, eval_check_feature
from kaggle_utils import reduce_mem_usage, move_feature
logger = logger_func()

2019-10-02 23:33:35,339 func.utils 347 [INFO]    [logger_func] start 


In [2]:
COLUMN_ID = 'TransactionID'
COLUMN_DT = 'TransactionDT'
COLUMN_TARGET = 'isFraud'
COLUMN_GROUP = 'DT-M'
COLUMNS_IGNORE = [COLUMN_ID, COLUMN_DT, COLUMN_TARGET, COLUMN_GROUP, 'is_train', 'date']

def filter_feature(path):
    if path.count(''):
        return True
    else:
        return False

# paths_train = glob('../submit/re_sub/50*_train.gz')
# paths_test  = glob('../submit/re_sub/50*_test.gz')
# paths_train += glob('../submit/re_sub/Tran*_train.gz')
# paths_test  += glob('../submit/re_sub/Tran*_test.gz')
# paths_train += glob('../submit/re_sub/is*_train.gz')
# paths_test  += glob('../submit/re_sub/is*_test.gz')

paths_train = glob('../submit/re_sub/*_train.gz')
paths_test  = glob('../submit/re_sub/*_test.gz')
# paths_train += glob('../submit/add_feature/*_train.gz')
# paths_test  += glob('../submit/add_feature/*_test.gz')

print(len(paths_train))
# sys.exit()
# paths_train += glob('../feature/valid_trush/528*uid2*_train.gz')
# paths_test  += glob('../feature/valid_trush/528*uid2*_test.gz')

# for path in paths_train:
#     if path.count('C14_ratio'):
#         paths_train.remove(path)
        
# for path in paths_test:
#     if path.count('C14_ratio'):
#         paths_test.remove(path)
    
# paths_train = glob('../feature/raw_use/*_train.gz')
# paths_test = glob('../feature/raw_use/*_test.gz')
# paths_train = [path for path in paths_train if filter_feature(path) ]
# paths_test = [path for path in paths_test if filter_feature(path) ]

# paths_train_feature = sorted(glob('../feature/org_use/*_train.gz'))
# paths_test_feature  = sorted(glob('../feature/org_use/*_test.gz'))

# paths_train_feature += sorted(glob('../feature/valid/*_train.gz'))
# paths_test_feature  += sorted(glob('../feature/valid/*_test.gz'))

# paths_train_feature += sorted(glob('../feature/kernel/*_train.gz'))
# paths_test_feature  += sorted(glob('../feature/kernel/*_test.gz'))

# paths_train_feature = sorted(glob('../feature/valid_use/*_train.gz'))
# paths_test_feature  = sorted(glob('../feature/valid_use/*_test.gz'))
paths_train_feature = []
paths_test_feature  = []

# df_train = reduce_mem_usage( parallel_load_data(paths_train) )
# df_test  = reduce_mem_usage( parallel_load_data(paths_test) )
df_train = parallel_load_data(paths_train)
df_test  = parallel_load_data(paths_test)
Y = df_train[COLUMN_TARGET]
df_train.drop(COLUMN_TARGET, axis=1, inplace=True)

1234


In [3]:
# data = pd.concat([df_train, df_test], axis=0)

# cols_categorical = [col for col in df_train.columns 
#                     if not col in COLUMNS_IGNORE
#                     and 
#                     (
#                     col.count('uid')
#                     or col.count('ugr')
#                     or col.count('60')
#                     )
#                    ]

# list_drop = []
# for col in tqdm(cols_categorical):
#     cnt = df_train[col].head(2000).value_counts().shape[0]
#     if cnt > 100:
#         list_drop.append(col)
#         continue
#     cnt = data[col].value_counts().shape[0]
#     if cnt > 200:
#         list_drop.append(col)
#     else:
#         data[col].fillna(-99999, inplace=True)
            
# cols_categorical = list(set(cols_categorical) -set(list_drop))
# print(len(cols_categorical))

# data = get_factorize_feature(data, cols_categorical)
# df_train = data.iloc[:len(df_train)]
# df_test  = data.iloc[len(df_train):]


# same_user_path = '../output/same_user_pattern/20190901_user_ids_share.csv'
# same_user_path = '../output/same_user_pattern/20190901_user_ids_share.csv'
# bear = pd.read_csv(same_user_path)
# bear = bear[[COLUMN_ID, 'predicted_user_id']]
# max_id = bear['predicted_user_id'].max()
# bear.loc[bear[bear['predicted_user_id'].isnull()].index, 'predicted_user_id'] = np.arange(
#     bear['predicted_user_id'].isnull().sum() ) + 1 + max_id
# bear['predicted_user_id'] =  bear['predicted_user_id'].astype('int')
# bear.shape

In [4]:
is_submit = [True, False][0]
n_splits = 5
set_type = 'new_set'

valid_paths_train = paths_train_feature[:]
valid_paths_test  = paths_test_feature[:]

#========================================================================
# pathの存在チェック。なぜかたびたびFileNotFoundErrorが起きるので,,,
#========================================================================
remove_paths = []
for trn_path, tes_path in zip(valid_paths_train, valid_paths_test):
    if os.path.exists(trn_path) and os.path.exists(tes_path):
        pass
    else:
        remove_paths.append(trn_path)
        remove_paths.append(tes_path)
for path in remove_paths:
    if path.count('train'):
        valid_paths_train.remove(path)
        print(f'remove {path}')
    elif path.count('test'):
        valid_paths_test.remove(path)
        print(f'remove {path}')

if len(valid_paths_train):
    df_feat_train = parallel_load_data(valid_paths_train)
    df_feat_test  = parallel_load_data(valid_paths_test)
    
    col_drops = eval_check_feature(df_feat_train, df_feat_test)
    
    tmp_train = df_train.join(df_feat_train)
    tmp_test = df_test.join(df_feat_test)
else:
    tmp_train = df_train
    tmp_test = df_test

#========================================================================
# Train Test で片方に存在しないFeatureを除外
#========================================================================
diff_cols = list(set(tmp_train.columns) - set(tmp_test.columns))

for col in list(set(diff_cols)):
    from_dir = 'valid'
    to_dir = 'valid_trush'
    move_feature([col], from_dir, to_dir)
tmp_train.drop(diff_cols, axis=1, inplace=True)
print(f"  * Diff Features: {len(diff_cols)}")

# same_user_path = '../output/same_user_pattern/0902__same_user_id__card_addr_pemail_M.csv'
# tmp_train = tmp_train.merge(bear[[COLUMN_ID, 'predicted_user_id']], how='inner', on=COLUMN_ID)
# COLUMN_GROUP = 'predicted_user_id'
# COLUMNS_IGNORE.append('predicted_user_id')

### DT-M
group_kfold_path = '../input/0908_ieee__DT-M_GroupKFold.gz'
group = read_pkl_gzip(group_kfold_path)
tmp_train[COLUMN_GROUP] = group

# tmp_train[COLUMN_GROUP] = tmp_train['528__ugr_uid3_Regist_date_agg_V95_137_mean_mean'].fillna(0)

#========================================================================
# Features elimination 
#==============================================================
# from scipy.stats import ks_2samp
# features_check = []
# columns_to_check = set(list(tmp_train)).difference(COLUMNS_IGNORE)
# for i in columns_to_check:
#     features_check.append(ks_2samp(tmp_test[i], tmp_train[i])[1])

# features_check = pd.Series(features_check, index=columns_to_check).sort_values() 
# features_discard = list(features_check[features_check==0].index)
# print(features_discard)
# tmp_train.drop(features_discard, axis=1, inplace=True)
# tmp_test.drop(features_discard, axis=1, inplace=True)

  * Diff Features: 0


In [ ]:
n_splits = 5
model_type = "cat"
params = {
#     'n_jobs': 96,
    'seed': 1208,
#     'subsample': 0.9,
#     'boosting_type':'Ordered',
    'num_boost_round':10000,
    'random_seed': 1208,
#     'has_time': True,
    'max_depth': 10,
    'loss_function': 'Logloss',
    'custom_loss':['AUC'],
#     'logging_level':'Silent',
    'task_type' : 'CPU',
    'early_stopping_rounds' : 100,
    'n_splits': n_splits,
    'model_type': model_type,
    'fold': ['stratified', 'group'][1],
    'verbose': 50,
}

if is_submit:
    pass
    params['learning_rate'] = 0.01
#     params['learning_rate'] = 0.07
#     params['learning_rate'] = 0.1
#     params["early_stopping_rounds"] = 100
    params["early_stopping_rounds"] = 50
#     params["early_stopping_rounds"] = 10

use_cols = [col for col in df_train.columns if col not in COLUMNS_IGNORE]
logger.info(f"* EXP: dataset {set_type} {tmp_train.shape} lr {params['learning_rate']} ")
            
if False:
#     sample = tmp_train.sample(10000)
#     sample[COLUMN_GROUP].value_counts()
    use_cols = [col for col in df_train.columns if col not in COLUMNS_IGNORE][:10]
    params['early_stopping_rounds'] = 1
    params['num_boost_round'] = 1

feim, _ = eval_train(
    logger,
    tmp_train,
    Y,
    tmp_test,
#     sample,
#     Y.head(10000),
#     tmp_test.head(1000),
    COLUMN_GROUP,
    model_type,
    params,
    is_adv=[True, False][1],
    is_viz=[True, False][1],
#     cols_categorical=cols_categorical
)
feim = list_result_feim[0]

2019-10-02 23:34:44,394 func.utils 33 [INFO]    [<module>] * EXP: dataset new_set (590540, 1234) lr 0.01  


0:	learn: 0.6756240	test: 0.6764498	best: 0.6764498 (0)	total: 1.77s	remaining: 4h 55m 31s
50:	learn: 0.2313863	test: 0.2380128	best: 0.2380128 (50)	total: 1m 10s	remaining: 3h 49m 13s
100:	learn: 0.1371518	test: 0.1368381	best: 0.1368381 (100)	total: 2m 17s	remaining: 3h 44m 15s
150:	learn: 0.1101899	test: 0.1068106	best: 0.1068106 (150)	total: 3m 24s	remaining: 3h 42m 11s
200:	learn: 0.0993151	test: 0.0949655	best: 0.0949655 (200)	total: 4m 32s	remaining: 3h 41m 10s
250:	learn: 0.0934283	test: 0.0892683	best: 0.0892683 (250)	total: 5m 41s	remaining: 3h 40m 46s
300:	learn: 0.0893930	test: 0.0863313	best: 0.0863313 (300)	total: 6m 48s	remaining: 3h 39m 31s
350:	learn: 0.0865129	test: 0.0840395	best: 0.0840395 (350)	total: 7m 56s	remaining: 3h 38m 13s
400:	learn: 0.0839848	test: 0.0822417	best: 0.0822417 (400)	total: 9m 3s	remaining: 3h 36m 59s
450:	learn: 0.0818648	test: 0.0812439	best: 0.0812430 (449)	total: 10m 11s	remaining: 3h 35m 47s
500:	learn: 0.0800398	test: 0.0804629	best: 0.0

2019-10-03 01:50:38,088 func.utils 168 [INFO]    [ieee_cv]   * Fold0 2017-12: 0.9318368508185453 | Bear's...PB:0.9165662153383537 PV:0.9153257212423997 All:0.9157662805569584 


0:	learn: 0.6745950	test: 0.6745815	best: 0.6745815 (0)	total: 1.57s	remaining: 4h 22m 1s
50:	learn: 0.2251395	test: 0.2316345	best: 0.2316345 (50)	total: 1m 8s	remaining: 3h 42m 37s
100:	learn: 0.1307960	test: 0.1415756	best: 0.1415756 (100)	total: 2m 14s	remaining: 3h 40m 28s
150:	learn: 0.1042225	test: 0.1182586	best: 0.1182586 (150)	total: 3m 21s	remaining: 3h 39m 9s
200:	learn: 0.0934345	test: 0.1099121	best: 0.1099121 (200)	total: 4m 27s	remaining: 3h 37m 42s
250:	learn: 0.0880127	test: 0.1060539	best: 0.1060539 (250)	total: 5m 34s	remaining: 3h 36m 18s
300:	learn: 0.0842831	test: 0.1035542	best: 0.1035542 (300)	total: 6m 41s	remaining: 3h 35m 29s
350:	learn: 0.0813941	test: 0.1015636	best: 0.1015636 (350)	total: 7m 47s	remaining: 3h 34m 19s
400:	learn: 0.0790828	test: 0.0999609	best: 0.0999609 (400)	total: 8m 54s	remaining: 3h 33m 17s
450:	learn: 0.0771875	test: 0.0986292	best: 0.0986292 (450)	total: 10m 1s	remaining: 3h 32m 7s
500:	learn: 0.0755148	test: 0.0975373	best: 0.09753

2019-10-03 04:52:15,036 func.utils 168 [INFO]    [ieee_cv]   * Fold1 2018-3: 0.9483098336330369 | Bear's...PB:0.9216978431961692 PV:0.9235016436923962 All:0.9234222452041446 


[  * Fold1 Validation-DT-M 2018-3: 101968] done in 10889 s
0:	learn: 0.6749340	test: 0.6751861	best: 0.6751861 (0)	total: 2.07s	remaining: 5h 45m
50:	learn: 0.2255372	test: 0.2315652	best: 0.2315652 (50)	total: 1m 11s	remaining: 3h 54m 3s
100:	learn: 0.1312113	test: 0.1402716	best: 0.1402716 (100)	total: 2m 20s	remaining: 3h 49m 7s
150:	learn: 0.1049741	test: 0.1160752	best: 0.1160752 (150)	total: 3m 28s	remaining: 3h 46m 43s
200:	learn: 0.0945616	test: 0.1070435	best: 0.1070435 (200)	total: 4m 37s	remaining: 3h 45m 8s
250:	learn: 0.0889802	test: 0.1025887	best: 0.1025887 (250)	total: 5m 46s	remaining: 3h 44m 17s
300:	learn: 0.0852284	test: 0.0998341	best: 0.0998341 (300)	total: 6m 55s	remaining: 3h 43m 13s
350:	learn: 0.0823183	test: 0.0978386	best: 0.0978386 (350)	total: 8m 4s	remaining: 3h 42m 9s
400:	learn: 0.0799851	test: 0.0961961	best: 0.0961961 (400)	total: 9m 13s	remaining: 3h 40m 51s
450:	learn: 0.0781961	test: 0.0949474	best: 0.0949474 (450)	total: 10m 21s	remaining: 3h 39m 